In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
BASE_PATH = "/gdrive/My Drive/Colab Notebooks/Innoplexus/"

In [3]:
import zipfile

print("Uncompressing the zip file...")
zip_ref = zipfile.ZipFile(BASE_PATH+"Innoplexus.zip", "r")
zip_ref.extractall("Innoplexus/")
zip_ref.close()
print("Finished...")

Uncompressing the zip file...
Finished...


In [0]:
DATA_PATH = "Innoplexus/Innoplexus/"

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [6]:
!pip install -q sklearn_crfsuite
import sklearn_crfsuite

from sklearn_crfsuite import metrics

    100% |████████████████████████████████| 757kB 19.5MB/s 


In [0]:
train = pd.read_csv(DATA_PATH+"train.csv")

test = pd.read_csv(DATA_PATH+"test.csv")

submission = pd.read_csv(DATA_PATH+"sample_submission.csv")

In [0]:
def func(x):
  words = (x['Word'].values.tolist())
  tags = (x['tag'].values.tolist())
  temp_list = []
  for word, tag in zip(words, tags):
    temp_list.append((word, tag))
  return temp_list

grouped = train.groupby('Sent_ID').apply(func)

word_tag = [wt for wt in grouped]

In [0]:
def doc2features(doc, i):
  
  word = str(doc[i][0])
  
  features={'word.word': word,
            'bias':1.0,
            'word.lower()':word.lower(),
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit()}
  
  if i > 0:
    prevword = str(doc[i-1][0])
    features['word.prevword'] = prevword
    features['-1:word.lower()'] = prevword.lower()
    features['-1:word.isupper()'] = prevword.isupper()
    features['-1:word.istitle()'] = prevword.istitle()
  else:
    features['BOS'] = True
        
  if i < len(doc)-1:
    nextword = str(doc[i+1][0])
    features['word.nextword'] = nextword
    features['+1:word.lower()'] = nextword.lower()
    features['+1:word.isupper()'] = nextword.isupper()
    features['+1:word.istitle()'] = nextword.istitle()
  else:
    features['EOS'] = True
  return features
  
def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]
  
def get_labels(doc):
    return [tag for (token,tag) in doc]

In [0]:
X = [extract_features(doc) for doc in word_tag]
y = [get_labels(doc) for doc in word_tag]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.1,
                                                    random_state=42)

In [0]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           c1=0.0325, c2=0.1, max_iterations=500,
                           all_possible_transitions=True)

In [12]:
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.0325, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=500,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [0]:
labels = crf.classes_

In [14]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9924639453687263

In [0]:
def func_test(x):
  words = (x['Word'].values.tolist())
  return words

grouped_test = test.groupby('Sent_ID').apply(func_test)

word_test = [wt for wt in grouped_test]

In [0]:
def doc2features_test(doc, i):
  
  word = str(doc[i])
  
  features={'word.word': word,
            'bias':1.0,
            'word.lower()':word.lower(),
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit()}
  
  if i > 0:
    prevword = str(doc[i-1])
    features['word.prevword'] = prevword
    features['-1:word.lower()'] = prevword.lower()
    features['-1:word.isupper()'] = prevword.isupper()
    features['-1:word.istitle()'] = prevword.istitle()
  else:
    features['BOS'] = True
        
  if i < len(doc)-1:
    nextword = str(doc[i+1])
    features['word.nextword'] = nextword
    features['+1:word.lower()'] = nextword.lower()
    features['+1:word.isupper()'] = nextword.isupper()
    features['+1:word.istitle()'] = nextword.istitle()
  else:
    features['EOS'] = True
  return features
  
def extract_features_test(doc):
    return [doc2features_test(doc, i) for i in range(len(doc))]

In [0]:
test_X = [extract_features_test(doc) for doc in word_test]

In [0]:
y_submit = crf.predict(test_X)

In [0]:
answers = []

for tags in y_submit:
  for tag in tags:
    answers.append(tag)

In [0]:
submission.tag = answers

In [0]:
attempt = 5
submission.to_csv("submission_{}.csv".format(attempt), index=False)